In [1]:
#!pip install jupysql pandas matplotlib duckdb-engine duckdb

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%sql duckdb:///:memory:

Connecting to 'duckdb:///:memory:'

In [3]:
%%sql
CREATE TABLE Riders             AS FROM "../data/riders.csv";
CREATE TABLE RiderTeams         AS FROM "../data/rider_teams.csv";
CREATE TABLE Managers           AS FROM "../data/managers.csv";
CREATE TABLE ManagerTeams       AS FROM "../data/manager_teams.csv";
CREATE TABLE ManagerTeamsCheapo AS FROM "../data/manager_cheapo_teams.csv";
CREATE TABLE PointsSystem       AS FROM "../data/points_system.csv";
CREATE TABLE Races              AS FROM "../data/races.csv";
CREATE TABLE Results_2023       AS FROM "../data/results/results_2023_full.csv";
CREATE TABLE Results_2024       AS FROM "../data/results/results_2024_full.csv";
CREATE TABLE Results_2025       AS FROM "../data/results/results_2025_full.csv";
CREATE TABLE CheapoBans         AS FROM "../data/cheapo_bans.csv";

Running query in 'duckdb:///:memory:'

Count


In [4]:
%%sql
CREATE OR REPLACE TABLE RaceResultsPoints AS
SELECT
     results.Race
    ,races.RaceName
    ,races.RaceCategory
    ,results.Stage_ID
    ,results.Stage_Date
    ,results.Stage_type
    ,results.Rider AS RiderName_PCS
    ,riders.RiderName_Zweeler
    ,results.Rank
    ,points.RacePoints AS Points
    ,CASE WHEN races.RaceName = 'Tour Down Under' THEN NULL Else points.RacePoints END AS CheapoPoints
FROM Results_2025 results
INNER JOIN Races races ON races.RaceName_PCS = results.Race
INNER JOIN Riders riders ON riders.RiderName_PCS = results.Rider
LEFT JOIN PointsSystem points ON 1 = 1
    AND (results.Stage_Type IS NULL OR results.Stage_Type <> 'TTT')
    AND points.RaceRank = TRY_CAST(results.Rank AS INTEGER) 
    AND CASE WHEN results.Stage_ID LIKE '%stage%' THEN REPLACE(races.RaceCategory, 'Tour', 'Stage') ELSE races.RaceCategory END = points.RaceCategory;

Running query in 'duckdb:///:memory:'

Count


In [8]:
%%sql
res <<
SELECT *
FROM RaceResultsPoints
WHERE RiderName_PCS LIKE '%Onley%'
ORDER BY Stage_ID ASC;

Running query in 'duckdb:///:memory:'

In [10]:
res_df = pd.DataFrame(res)

In [12]:
res_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>RACE</th>\n      <th>RaceName</th>\n      <th>RaceCategory</th>\n      <th>STAGE_ID</th>\n      <th>STAGE_DATE</th>\n      <th>STAGE_TYPE</th>\n      <th>RiderName_PCS</th>\n      <th>RiderName_Zweeler</th>\n      <th>RANK</th>\n      <th>Points</th>\n      <th>CheapoPoints</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>tour-down-under</td>\n      <td>Tour Down Under</td>\n      <td>Tour Cat. 2</td>\n      <td>gc</td>\n      <td>2025-01-26</td>\n      <td>None</td>\n      <td>Onley Oscar</td>\n      <td>Onley, Oscar</td>\n      <td>4</td>\n      <td>60.0</td>\n      <td>NaN</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>uae-tour</td>\n      <td>UAE Tour</td>\n      <td>Tour Cat. 2</td>\n      <td>gc</td>\n      <td>2025-02-23</td>\n      <td>None</td>\n      <td>Onley Oscar</td>\n      <td>Onley, Oscar</td>\n      <td>5</td>\n      <td